In [1]:
from google.colab import drive
drive.mount("/content/drive")
%cd "/content/drive/My Drive/Colab Notebooks/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [3]:
dataset = pd.read_csv('loan_defaulter_dataset.csv')
dataset.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,11-Dec,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-85,1.0,3.0,0.0,13648.0,83.70%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,15-Jan,171.62,16-Dec,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,11-Dec,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-99,5.0,3.0,0.0,1687.0,9.40%,4.0,f,0.00,0.00,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,13-Apr,119.66,16-Oct,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,11-Dec,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,1-Nov,2.0,2.0,0.0,2956.0,98.50%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.0,0.00,14-Jun,649.91,16-Dec,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,11-Dec,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-96,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.0,0.00,15-Jan,357.48,16-Apr,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,11-Dec,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-96,0.0,15.0,0.0,27783.0,53.90%,38.0,f,73.71,73.71,3987.860000,3987.86,2926.29,1061.57,0.00,0.0,0.00,16-Dec,67.79,16-Dec,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [0]:
profile=pandas_profiling.ProfileReport(dataset)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


In [0]:
#Replacing a the word 'months' by nothing in the 'term' coloumb
dataset['term'] = dataset['term'].str.replace('months','')
#Replacing a the word '%' by nothing in the 'int_rate' coloumb
dataset['int_rate'] = dataset['int_rate'].str.replace('%','')

In [5]:
#Dropping coloumn 'id' and 'member_id' because it doesnt make sense
dataset=dataset.drop(columns=['id', 'member_id'])
dataset

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,5000.0,4975.0,36,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,11-Dec,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-85,1.0,3.0,0.0,13648.0,83.70%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,15-Jan,171.62,16-Dec,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,2500.0,2500.0,60,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,11-Dec,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-99,5.0,3.0,0.0,1687.0,9.40%,4.0,f,0.00,0.00,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,13-Apr,119.66,16-Oct,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,2400.0,2400.0,36,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,11-Dec,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,1-Nov,2.0,2.0,0.0,2956.0,98.50%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.0,0.00,14-Jun,649.91,16-Dec,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,10000.0,10000.0,36,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,11-Dec,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-96,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.0,0.00,15-Jan,357.48,16-Apr,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,3000.0,3000.0,3000.0,60,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,11-Dec,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-96,0.0,15.0,0.0,27783.0,53.90%,38.0,f,73.71,73.71,3987.860000,3987.86,2926.29,1061.57,0.00,0.0,0.00,16-Dec,67.79,16-Dec,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42531,3500.0,3500.0,225.0,36,10.28,113.39,C,C1,NaN,< 1 year,RENT,180000.0,Not Verified,7-Jun,Does not meet the credit policy. Status:Fully ...,n,other,Wedding coming up,100xx,NY,10.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,f,0.00,0.00,3719.431070,239.11,3500.00,219.43,0.00,0.0,0.00,8-Mar,0.00,13-Feb,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN
42532,1000.0,1000.0,0.0,36,9.64,32.11,B,B4,Halping hands company inc.,< 1 year,RENT,12000.0,Not Verified,7-Jun,Does not meet the credit policy. Status:Fully ...,n,other,delight,021xx,MA,10.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,f,0.00,0.00,1155.600899,0.00,1000.00,155.60,0.00,0.0,0.00,10-Jun,32.41,14-Sep,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN
42533,2525.0,2525.0,225.0,36,9.33,80.69,B,B3,NaN,< 1 year,RENT,110000.0,Not Verified,7-Jun,Does not meet the credit policy. Status:Fully ...,n,other,Car repair bill,100xx,NY,10.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,f,0.00,0.00,2904.498829,258.82,2525.00,379.50,0.00,0.0,0.00,10-Jun,82.03,7-May,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN
42534,6500.0,6500.0,0.0,36,8.38,204.84,A,A5,NaN,< 1 year,NONE,NaN,Not Verified,7-Jun,Does not meet the credit policy. Status:Fully ...,n,other,Buying a car,100xx,NY,4.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,f,0.00,0.00,7373.904962,0.00,6500.00,873.90,0.00,0.0,0.00,10-Jun,205.32,7-Aug,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN


In [0]:
# count=0
# for i in range (42536):
#   if dataset['loan_amnt'][i] != dataset['funded_amnt'][i]:
#     print("Total score for %s is %s" % ( str(dataset['loan_amnt'][i]), str(dataset['funded_amnt'][i]) ))

# print(count)

# **DROPPING HIGH PERCENTAGE OF NULL or  N/A or ZEROS**

In [0]:
coloumn_names=dataset.columns.values.tolist()

In [7]:
threshold_for_null_and_NA= 10

for col in coloumn_names:
  percentage_missing= (  (  dataset[col].isnull().sum() +   dataset[col].isna().sum() + dataset[col].isin([0]).sum()  )  / 42536 ) *100

  if percentage_missing > threshold_for_null_and_NA:
    print (col)

emp_title
delinq_2yrs
inq_last_6mths
pub_rec
out_prncp
out_prncp_inv
total_rec_late_fee
recoveries
collection_recovery_fee
collections_12_mths_ex_med
acc_now_delinq
chargeoff_within_12_mths
delinq_amnt
pub_rec_bankruptcies
tax_liens


In [8]:
dataset=dataset.drop(columns=['emp_title','delinq_2yrs','pub_rec','out_prncp','out_prncp_inv','total_rec_late_fee','recoveries','collection_recovery_fee','collections_12_mths_ex_med','acc_now_delinq','chargeoff_within_12_mths','delinq_amnt','pub_rec_bankruptcies','tax_liens'])
dataset

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,earliest_cr_line,inq_last_6mths,open_acc,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,policy_code,application_type
0,5000.0,5000.0,4975.0,36,10.65,162.87,B,B2,10+ years,RENT,24000.0,Verified,11-Dec,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,Jan-85,1.0,3.0,13648.0,83.70%,9.0,f,5863.155187,5833.84,5000.00,863.16,15-Jan,171.62,16-Dec,1.0,INDIVIDUAL
1,2500.0,2500.0,2500.0,60,15.27,59.83,C,C4,< 1 year,RENT,30000.0,Source Verified,11-Dec,Charged Off,n,car,bike,309xx,GA,1.00,Apr-99,5.0,3.0,1687.0,9.40%,4.0,f,1014.530000,1014.53,456.46,435.17,13-Apr,119.66,16-Oct,1.0,INDIVIDUAL
2,2400.0,2400.0,2400.0,36,15.96,84.33,C,C5,10+ years,RENT,12252.0,Not Verified,11-Dec,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,1-Nov,2.0,2.0,2956.0,98.50%,10.0,f,3005.666844,3005.67,2400.00,605.67,14-Jun,649.91,16-Dec,1.0,INDIVIDUAL
3,10000.0,10000.0,10000.0,36,13.49,339.31,C,C1,10+ years,RENT,49200.0,Source Verified,11-Dec,Fully Paid,n,other,personel,917xx,CA,20.00,Feb-96,1.0,10.0,5598.0,21%,37.0,f,12231.890000,12231.89,10000.00,2214.92,15-Jan,357.48,16-Apr,1.0,INDIVIDUAL
4,3000.0,3000.0,3000.0,60,12.69,67.79,B,B5,1 year,RENT,80000.0,Source Verified,11-Dec,Current,n,other,Personal,972xx,OR,17.94,Jan-96,0.0,15.0,27783.0,53.90%,38.0,f,3987.860000,3987.86,2926.29,1061.57,16-Dec,67.79,16-Dec,1.0,INDIVIDUAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42531,3500.0,3500.0,225.0,36,10.28,113.39,C,C1,< 1 year,RENT,180000.0,Not Verified,7-Jun,Does not meet the credit policy. Status:Fully ...,n,other,Wedding coming up,100xx,NY,10.00,NaN,NaN,NaN,0.0,NaN,NaN,f,3719.431070,239.11,3500.00,219.43,8-Mar,0.00,13-Feb,1.0,INDIVIDUAL
42532,1000.0,1000.0,0.0,36,9.64,32.11,B,B4,< 1 year,RENT,12000.0,Not Verified,7-Jun,Does not meet the credit policy. Status:Fully ...,n,other,delight,021xx,MA,10.00,NaN,NaN,NaN,0.0,NaN,NaN,f,1155.600899,0.00,1000.00,155.60,10-Jun,32.41,14-Sep,1.0,INDIVIDUAL
42533,2525.0,2525.0,225.0,36,9.33,80.69,B,B3,< 1 year,RENT,110000.0,Not Verified,7-Jun,Does not meet the credit policy. Status:Fully ...,n,other,Car repair bill,100xx,NY,10.00,NaN,NaN,NaN,0.0,NaN,NaN,f,2904.498829,258.82,2525.00,379.50,10-Jun,82.03,7-May,1.0,INDIVIDUAL
42534,6500.0,6500.0,0.0,36,8.38,204.84,A,A5,< 1 year,NONE,NaN,Not Verified,7-Jun,Does not meet the credit policy. Status:Fully ...,n,other,Buying a car,100xx,NY,4.00,NaN,NaN,NaN,0.0,NaN,NaN,f,7373.904962,0.00,6500.00,873.90,10-Jun,205.32,7-Aug,1.0,INDIVIDUAL


In [11]:
dataset['issue_d']

0        11-Dec
1        11-Dec
2        11-Dec
3        11-Dec
4        11-Dec
          ...  
42531     7-Jun
42532     7-Jun
42533     7-Jun
42534     7-Jun
42535     7-Jun
Name: issue_d, Length: 42536, dtype: object

In [0]:
plt.figure()
plt.suptitle("Seaborn heatmap example to display 2d percentage values", color='m')
sns.set(font_scale=0.8)

ax = sns.heatmap(dataset, linewidths=.5,  fmt="", annot_kws={"size": 12}, vmin=0, vmax=100)
ax.set(xlabel='x label', ylabel='y label')

plt.show()